## LChain

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
ids = vector_store.add_documents(documents=all_splits)

In [ ]:
!pip install -qU langchain-ollama

In [ ]:
# Igual se lo meto en manual... no estoy seguro que me aporta este método si luego proveo el embedding para búsqueda yo mismo...
embeddings = OllamaEmbeddings(model="bge-m3")

In [ ]:
results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

## ChromaDB + Ollama

In [ ]:
#!pip install chromadb==0.5.23
#!pip install ollama==0.4.7


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os
import sys
import pickle
import chromadb
import torch
from chromadb import Documents, EmbeddingFunction, Embeddings
from ollama import Client

# Add "src" path to Python path
sys.path.append(os.path.abspath("../src"))

# Import custom embedding function
from embedding_utils import get_ollama_embedding
from sage_utils import get_new_sage_embedding


In [5]:
# Load graph
with open("../data/MultiHop_graph_w_sage100_embeddings.pkl", "rb") as f:
    G = pickle.load(f)

In [11]:
G.nodes["100_chunk_0"].keys()

dict_keys(['type', 'category', 'source', 'author', 'text', 'embedding', 'SAGE_embedding'])

In [ ]:
# Instantiate client
chroma_client = chromadb.Client()

# Create a collection
collection_semrag = chroma_client.create_collection(name="semantic_rag")

# Iterate over the nodes and filter those with type "chunk"
for node_id in G.nodes:
    node_data = G.nodes[node_id]
    if node_data.get("type") == "chunk":
        doc_text = node_data.get("text", "")
        doc_embedding = node_data.get("embedding", None)
        
        # Ensure that doc_embedding is a list (or array) of floats
        # Add the node to the collection by specifying IDs, documents, and embeddings
        collection_semrag.add(
            ids=[str(node_id)],         # The ID will be the node's ID (converted to a string)
            documents=[doc_text],         # The node's text
            embeddings=[doc_embedding]    # The precomputed embedding you already have
        )

In [ ]:
# Suppose you have a query embedding 'query_embedding'
query = "Some query here"
query_bgem3 = get_ollama_embedding(query)["embedding"]
query_sage = get_new_sage_embedding(torch.tensor(query_bgem3, dtype=torch.float32).reshape(1024))

In [ ]:
# Query the collection
results = collection_semrag.query(
    query_embeddings=[query_sage],
    n_results=5
)

print(results)

# WIP
processed_results = results

In [ ]:
## Custom ebeddings with CHromaDB
#from chromadb import Documents, EmbeddingFunction, Embeddings
#
#class MyEmbeddingFunction(EmbeddingFunction):
#    def __call__(self, input: Documents) -> Embeddings:
#        # embed the documents somehow
#        return embeddings

In [ ]:
## Querying collection with custom embeddings
#collection.query(
#    query_embeddings=[[11.1, 12.1, 13.1],[1.1, 2.3, 3.2], ...],
#    n_results=10,
#    where={"metadata_field": "is_equal_to_this"},
#    where_document={"$contains":"search_string"}
#)

In [ ]:

client = Client(
  host='http://localhost:11434',
  #headers={'x-some-header': 'some-value'}
)
response = client.chat(model='hf.co/unsloth/QwQ-32B-GGUF:Q6_K', messages=[
  {
    'role': 'user',
    'content': f"""
You are an expert assistant whose task is to answer the user's question using the provided context. Use the context to support your answer and ensure that your response is clear, accurate, and concise.

Context:
{processed_results}

User Question:
{query}

Answer:
""",
  },
])

print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)